In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os

## I) Loading Data Into Pandas DataFrames and Inspecting It

To load data into a Pandas Dataframe, you can use the `pandas.read_*` suite of functions. Common examples are.
- `pandas.read_csv()` to read from a comma separated values file
- `pandas.read_sql()` to read directly from a SQL database (either table by name or SQL query constructed as a string)
- `pandas.read_table()` to read from a more general delimited tabular file (could be comma, tab, space, `'|'`, or otherwise delimited)
- `pandas.read_json()` to read from a .json file, but pandas will expect the JSON data to have a particular structure
- `pandas.read_parquet()` to read from an Apache Parquet columnar store file

Below, we use `pandas.read_parquet()` to read from an [Apache Parquet](https://parquet.apache.org/) data file

In [ ]:
# example: loading the synthetic patients file simulated for the state of Massachusetts
patients_ma = pd.read_parquet('https://github.com/expmed/arch_workshop_core_python_skills_ws13/raw/refs/heads/main/data/output_ma_small/parquet/patients.parquet')

To inspect the data in a DataFrame visually, we can just write the name of our dataframe and evaluate it. Or if we want a shorter summary, we can use the `DataFrame.head()` method to only show the top n rows

In [ ]:
# display the first 5 rows
patients_ma.head(5)

We can also display the names of all columns/variables in the DataFrame, and their types, by accessing the `.columns` and `.dtypes` attributes

In [ ]:
# display the columns in the DataFrame
patients_ma.columns

In [ ]:
# display the columns and their datatypes
patients_ma.dtypes

We can also access the data in a particular column (or set of columns) by providing the column name(s) in square brackets
- When we access a single column, the type of object we get back is known as a Pandas `Series`. Think of this like a column of data
- When we access multiple columns of data using a list of names, such as `['Column1', 'Column2', 'Column3', ...]` we get back another `DataFrame`

In [ ]:
# accessing the data in a particular column
patients_ma['FIRST']

In [ ]:
type(patients_ma['FIRST'])

In [ ]:
# accessing the data in a set of columns
patients_ma[['FIRST', 'MIDDLE', 'LAST']]


In [ ]:
type(patients_ma[['FIRST', 'MIDDLE', 'LAST']])

In [ ]:
# columns can also be accesssed using the dot notation like with attributes
patients_ma.FIRST

## II) Basic Data Manipulation with DataFrames

### II.a) Stitching (Concatenating) Data Together
- In Section I, we loaded in patient demographics data for the state of Massachussets
- We also have simulated data for the states of Hawaii, Texas, and Washington
- If we want to work with a single dataset of all patients in a single DataFrame, we can use the building `pandas.concat()` function
- `pandas.concat()` can take in a bracketed list of dataframes, such as `[df1, df2, df3,...]` and concatenate them together row-wise into a single table
- The only requirement (or recommendation) is that all dataframes to be appended in this way have the same schema (names, types, and number of columns)

In [ ]:
# helper function for loading data from the Github repository
def load_data_from_github(filename):
    url = os.path.join('https://github.com/expmed/arch_workshop_data_wrangling1_ws9/raw/refs/heads/main/data', filename)
    return pd.read_parquet(url)

In [ ]:
# load in the patients data for the other states
patients_hi = load_data_from_github('output_hi_small/parquet/patients.parquet')
patients_wa = load_data_from_github('output_wa_small/parquet/patients.parquet')
patients_tx = load_data_from_github('output_tx_small/parquet/patients.parquet')

# now combine the four DataFrames into one
patients = pd.concat([patients_ma, patients_hi, patients_wa, patients_tx], ignore_index=True)

### II.b) Counting

Suppose we want to count the number of patients by GENDER in our dataset
We can do so in two different ways
1. We can use the `Series.value_counts()` to count the distinct values in a given column/`Series`
2. We can use the `DataFrame.groupby()` method to group rows by a particular column (or set of columns) and then apply different aggregations

In [ ]:
# count the number of male and female patients
patients['GENDER'].value_counts()

In [ ]:
# alternative approach using the groupby method, which takes in a column name, and then we apply an aggregate function
patients.groupby('GENDER').size()

### II.c) Cross tabulations/Contingency Tables

Suppose now that we want to count the number of patients by both `GENDER` and `RACE`. We can do so in two different ways
1. Using the `pandas.crosstab()` method, which takes in two different columns/`Series` to be cross-tabulated
2. Using the `DataFrame.pivot_table()` method, which takes in the column names as either `index=` or `columns=` arguments, and an aggregate function

`pandas.crosstab()` is arguably the cleaner/nicer interface, since it automatically fills NAs and converts to integers

In [ ]:
# count patients by gender and race
pd.crosstab(patients['GENDER'], patients['RACE'])

In [ ]:
# alternative method using df.pivot_table(), but need to provide an aggregate function
patients.pivot_table(index='GENDER', columns='RACE', aggfunc='size')

However, `DataFrame.pivot_table()` is more versatile, since we can also give the name of a column of values to be aggregated \
and we can do different kinds of aggregations (e.g., averaging, summing, min, max, etc.)

In [ ]:
# computing the average healthcare expenses by gender and race
patients.pivot_table(values='HEALTHCARE_EXPENSES', index='GENDER', columns='RACE', aggfunc='mean')

In [ ]:
# computing the total healthcare expenses by gender and race
patients.pivot_table(values='HEALTHCARE_EXPENSES', index='GENDER', columns='RACE', aggfunc='sum')

### II.d) Descriptive Statistics
If we want to compoute descriptive statistics for particular variables/columns, there are a variety of built-in methods on `Series` as well as built in to the NumPy package

In [ ]:
# compute the average healthcare expenses
patients['HEALTHCARE_EXPENSES'].mean()

In [ ]:
# accessing a larger suite of descriptive statistics
patients['HEALTHCARE_EXPENSES'].describe()

### II.e) Descriptive Statistics with Numpy

In [ ]:
# compute the average healthcare expenses
np.mean(patients['HEALTHCARE_EXPENSES'])

In [ ]:
# compute the standard deviation of healthcare expenses
np.std(patients['HEALTHCARE_EXPENSES'])

In [ ]:
# we can also call methods directly on columns/pandas series
patients['HEALTHCARE_EXPENSES'].std()

Quick Question: Why did Numpy's std() give a different (smaller) value for the standard deviation than Pandas?

### II.f) Filtering DataFrames
If we want to extract data for specific subsets of records based on the values assigned to certain variables, we can use logical indexing
- If we want to extract all records with a certain value in a column, we can write `df[df['column_name'] == value]`
- We can use typical boolean operators for filtering such as `>`, `<`, `>=`, `<=`, etc.
If we want to chain logical filters we use the bitwise operators:
- `&` for logical `AND`
- `|` for logical `OR`
We need to be careful about order of operations here, since `|` and `&` have higher precedence than `==`, `<`, etc. in Python


In [ ]:
# Selecting only patients who are female
females = patients[patients['GENDER'] == 'F'] # or patients.query('GENDER == "F"')

In [ ]:
females

In [ ]:
# get patients born after 1990
patients[patients['BIRTHDATE'] >= '1990-01-01']

In [ ]:
# get patients who are female AND born after 1990
patients[
    (patients['BIRTHDATE'] >= '1990-01-01') &
    (patients['GENDER'] == 'F')
]

Pandas also supports a `DataFrame.query()` method that has a somewhat more succinct, if not slightly more confusing syntax for filtering purposees

In [ ]:
# Same as the example above, but using the query method
patients.query('BIRTHDATE >= "1990-01-01" and GENDER == "F"')

### II.g) Sorting
We can also sort the records/rows in a dataframe by a particular column or set of columns using `DataFrame.sort_values()`
- This method takes in an optional keyword argument `ascending=True|False` which can be used for ascending/decreasing order of the sort

In [ ]:
# get patients born after 1990, and sort by birthdate
patients[patients['BIRTHDATE'] >= '1990-01-01'].sort_values(by='BIRTHDATE')

In [ ]:
# do the same, but sort in decreasing order by birthdate
patients.query('BIRTHDATE >= "1990-01-01"').sort_values(by='BIRTHDATE', ascending=False)

### II.h) Assigning New Columns/Variables
If we want to assign a new column/variable to the dataframe, we can do this by either
- Writing `df[<column_name>] = <values>`, which changes/modifies the DataFrame in place
- Using the `DataFrame.assign()` method, which by default returns a copy of the original dataframe with the new column added
- `DataFrame.assign()` is quite a bit more flexible, easier to read, and easier to reason about (once you get comfortable with it), \
and it supports a higher order functional approach to specifying the values that get assigned 

In [ ]:
# assign a full name column
patients['FULLNAME'] = patients['FIRST'] + ' ' + patients['MIDDLE'] + ' ' + patients['LAST']

In [ ]:
patients['FULLNAME']

In [ ]:
# same as above, but using the asssign() method
patients.drop(columns=['FULLNAME'], inplace=True, errors='ignore') # drop the column if it exists
patients.assign(
    FULLNAME=lambda x: x['FIRST'] + ' ' + x['MIDDLE'] + ' ' + x['LAST']
)[['FULLNAME', 'FIRST', 'MIDDLE', 'LAST']]

### II.i) Filling in Missing Values (NaNs)
We can also use Pandas to work with missing data, filtering rows/columns with missing data, and imputing missing values \
We can inspect which values in a DataFrame are missing by calling `DataFrame.isnull()` \
If we want to fill in missing values, we can use `DataFrame.fillna()` or `Series.fillna()`

In [ ]:
patients.isnull()

Here we simulate a 10% missing values rate in the HEALTHCARE_EXPENSES column by randomly overwriting 10% of the values with `np.nan`
- `np.nan` is a built in special floating point sigil value provided by numpy and used to signify a missing value

In [ ]:
# simulate some missing values in the healthcare expenses column
# NOTE: DataFrame.loc[] is used to access a group of rows and columns by labels or a logical array
# NOTE: DataFrame.sample() is used to randomly sample rows from the DataFrame
# NOTE: DataFrame.index is used to access the index (row labels) of the DataFrame
np.random.seed(42) # for reproducibility
patients.loc[patients.sample(frac=0.1).index, 'HEALTHCARE_EXPENSES'] = np.nan
patients['HEALTHCARE_EXPENSES'].isnull().sum() / len(patients) * 100 # percentage of missing values

In [ ]:
# now if we wanted to fill in the missing values with the mean
patients['HEALTHCARE_EXPENSES'].fillna(patients['HEALTHCARE_EXPENSES'].mean()).isnull().sum()

In [ ]:
patients['HEALTHCARE_EXPENSES'].fillna(patients['HEALTHCARE_EXPENSES'].mean()).describe()

In [ ]:
patients['HEALTHCARE_EXPENSES'].describe()

### II.j) Handling DateTime Variables
By default, most data representing dates, times, and datetimes will be loaded by Pandas as plain text \
In order to treat these as actual date objects and perform calculations, we need to explicitly convert the columns \
to a `datetimeN` datatype. In pandas, we can use the `pd.to_datetime()` function to convert a series to a datetime type.



In [ ]:
# by default, the BIRTHDATE column is of type object (plaintext), but we can convert it to datetime
patients['BIRTHDATE']

In [ ]:
pd.to_datetime(patients['BIRTHDATE'])

In [ ]:
# we can also get todays date using pandas
pd.to_datetime('today')

In [ ]:
# we can easily do computations, such as the differenced between to dates
# what we get back from the computation is a series of timedelta objects
pd.to_datetime('today') - pd.to_datetime(patients['BIRTHDATE'])

## III) Visualizing Distributions with Pandas, Matplotlib, and Seaborn
The Matplotlib and Seaborn packages support a wide array of built-in chart types for creating figures/visualizations. \
Here we will just briefly look at some of the functionality. Pandas also has a number of built-in methods that can be \
called directly on `DataFrames` and `Series`, which use Matplotlib under the hood.

- Above, we imported the `pyplot` submodule from `matplotlib` with `from matplotlib import pyplot as plt`, so that we can just write `plt.<some_method>()` to use plotting functionality
- Similarly, we aliased `seaborn` as `import seaborn as sns`

In [ ]:
# Plotting the distribution of income as a histogram
fig, ax = plt.subplots(dpi=150)
patients['INCOME'].hist(bins=20, ax=ax)

In [ ]:
# alternative using matplotlib directly
plt.hist(patients['INCOME'], bins=20)
plt.show()

### III.a) Grouped Histograms
When creating histograms with `DataFrame.hist()`, we can also generate grouped histograms by providing a column name in the `by=` keyword argument

In [ ]:
# plotting the distributions of healthcare expenses by gender
fig, ax = plt.subplots(1, 2, dpi=150)
patients.hist('HEALTHCARE_EXPENSES', by='GENDER', bins=20, ax=ax)

### III.b) Box Plots and Violin Plots with Seaborn
The Seaborn package builds on top of Matplotlib with a nice set of customizable aesthetic defaults, and a somewhat extended suite of statistical charts compared to what is offered in vanilla Matplotlib

In [ ]:
# Alternative approaches to visualizing distributions with seaborn
sns.boxplot(patients, x='GENDER', y='HEALTHCARE_EXPENSES')

In [ ]:
sns.boxplot(patients, x='RACE', y='HEALTHCARE_EXPENSES', hue='GENDER')

In [ ]:
sns.violinplot(patients, x='GENDER', y='HEALTHCARE_EXPENSES')

In [ ]:
sns.violinplot(patients, x='RACE', hue='GENDER', y='HEALTHCARE_EXPENSES')

## IV) Quick Exercises

### 1. What is the median income of patients born after January 1st 1995

In [ ]:
# Write and run your solution here

### 2. Count the number of patients in each COUNTY

In [ ]:
# Write and run your solution here

### 3. What is the mean age (in years) of all patients?
<details>
    <summary>Hint 1</summary>
    <h4>You can convert BIRTHDATE to the datetime data type using <code>pd.to_datetime()</code></h4>
</details>
<details>
    <summary>Hint 2</summary>
    <h4>You can get today's date with <code>pd.to_datetime("today")</code></h4>
</details>
<details>
    <summary>Hint 3</summary>
    <h4>You can use the <code>.dt.days</code> method on a datetime column/series or datetime value to get the value of the date(s) expressed in days</h4>
</details>


In [ ]:
# Write and run your solution here

### 4. Count the number of patients of each RACE who are over 60
<details>
    <summary>Hint</summary>
    <h4>If you didn't save the AGE that we computed in the last exercise to a column/variable, do that first</h4>
</details>

In [ ]:
# Write and run your solution here

### 5. Generate a grouped histogram of income distributions by 10 year AGE cohorts


In [ ]:
# create an AGE_BIN column
patients['AGE_BIN'] = pd.cut(patients['AGE'], np.arange(0, 120, 10), include_lowest=True)

fig, ax = plt.subplots(3, 4, dpi=150, figsize=(8, 6))

# Flatten the axes array
ax = ax.flatten()

# Count the number of unique AGE_BINs
num_groups = patients['AGE_BIN'].nunique()

# insert your plotting code here, passing in ax=ax[:num_groups] as a keyword argument to the correct plot function

### 6. Plot INCOME vs AGE for all patients
<details>
    <summary>Hint</summary>
    <p>The pandas documentation for <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.scatter.html">DataFrame.plot.scatter</a> might prove useful</p>
</details>

In [ ]:
# Write and run your solution here